# Locality Reduction

In [65]:
from sympy import *
import re

sigmas = symbols('sigma1:5')

In [94]:
def reduce_positive_term_locality(sigmas, ancilla_symbol):
    sigma_np1 = ancilla_symbol
    res = 7 + prod(sigmas[2:]) - 3 * sigmas[0] - 3 * sigmas[1] + 6*sigma_np1 + 2 * prod(sigmas[2:]) * sigma_np1 - sigmas[0] * prod(sigmas[2:]) - sigmas[1] *prod(sigmas[2:]) - 4 * sigmas[0] * sigma_np1 - 4 * sigmas[1] * sigma_np1 + prod(sigmas[0:2])
    return res

def reduce_negative_term_locality(sigmas, ancilla_symbol):
    sigma_np1 = ancilla_symbol
    res = 5 - prod(sigmas[2:]) - sigmas[0] - sigmas[1] + 2 * sigma_np1 - 2 * prod(sigmas[2:]) * sigma_np1 + sigmas[0] * prod(sigmas[2:]) + sigmas[1] * prod(sigmas[2:]) - 4 * sigmas[0] * sigma_np1 - 4 * sigmas[1] * sigma_np1 + 3 * sigmas[0] * sigmas[1]
    return res

def separate_to_coeff_and_symbols(expr):
    syms = []
    coeffs = []
    for elem in Mul.make_args(expr):
       if type(elem) == Symbol:
           syms.append(elem)
       else:
           coeffs.append(elem)
    if coeffs:
       coeff = coeffs[0]
    else:
       coeff = 1
    return coeff, syms


def reduce_one_locality(expr, variable_name, maximum_index, to_locality):
    next_variable_index = maximum_index + 1
    reduced_expr = 0
    current_locality = 0
    for term in Add.make_args(expr):
        coeff, syms = separate_to_coeff_and_symbols(term)
        next_ancilla = Symbol(f'{variable_name}{next_variable_index}')
        next_variable_index += 1
        current_locality = max(current_locality, len(syms))
        if current_locality <= to_locality:
            continue
        if coeff > 0 :
            this_reduced_expr = reduce_positive_term_locality(syms, next_ancilla)
        else:
            coeff *= -1
            this_reduced_expr = reduce_negative_term_locality(syms, next_ancilla)
        reduced_expr += coeff * this_reduced_expr
    next_maximum_index = next_variable_index - 1
    reduced_locality = current_locality - 1
    return reduced_expr, next_maximum_index, reduced_locality


def reduce_locality(expr, variable_name, maximum_index, to_locality=2):
    reduced_expr, next_maximum_index, locality = reduce_one_locality(expr, variable_name, maximum_index, to_locality)
    print(locality)
    if locality < to_locality:
        return expr
    elif locality == to_locality:
        return reduced_expr
    else:
        return reduce_locality(reduced_expr, variable_name, next_maximum_index)
expr = prod(symbols("sigma0:4"))
reduced_expr, total_variables_count, locality = reduce_one_locality(expr, variable_name='sigma', maximum_index=3, to_locality=3)
print("Variables count ", total_variables_count, ", Locality ", locality)
reduced_expr

Variables count  4 , Locality  3


sigma0*sigma1 - sigma0*sigma2*sigma3 - 4*sigma0*sigma4 - 3*sigma0 - sigma1*sigma2*sigma3 - 4*sigma1*sigma4 - 3*sigma1 + 2*sigma2*sigma3*sigma4 + sigma2*sigma3 + 6*sigma4 + 7

In [95]:
reduce_locality(expr, variable_name='sigma', maximum_index=3, to_locality=2)

3
2


-4*sigma0*sigma13 + 3*sigma0*sigma2 + sigma0*sigma3 - sigma0 - 4*sigma1*sigma14 + 3*sigma1*sigma2 + sigma1*sigma3 - sigma1 - 4*sigma13*sigma2 - 2*sigma13*sigma3 + 2*sigma13 - 4*sigma14*sigma2 - 2*sigma14*sigma3 + 2*sigma14 - 8*sigma15*sigma2 - 8*sigma15*sigma3 + 4*sigma15*sigma4 + 12*sigma15 + 4*sigma2*sigma3 - 2*sigma2*sigma4 - 8*sigma2 - 2*sigma3*sigma4 - 8*sigma3 + 2*sigma4 + 24

In [ ]:


def reconstruct_interaaction_tensors(hamiltonian_expr):
    pass
    # indices = []
    # for sym in syms:
    #     name = sym.name
    #     match = re.match(r"sigma(\d+)", name)
    #     index = int(match.groups(0)[0])
    #     indices.append(index)
    #
    # print(coeffs, indices)